# Exercise 8: Unsupervised Learning - Dimensionality Reduction
## Theory
### Task 1: LLE - optimizing reconstruction loss $l(W) = \sum_{i=1}^t \lVert x_i - \sum_{j=1}^K W_{ij}x_j\rVert^2$
---
The constrained weights that best reconstruct eacht data point from its neighbors can be computed in closed form.

First, see Appendix A, page 9 in the [original LLE paper](http://www.cs.columbia.edu/~jebara/6772/papers/lleintro.pdf) and familiarize yourself with equations (3) and (4).

Then the solution of the given least squares problem can be computed by means of the Lagrangian function, that we presented in the lecture. For that we rephrase the problem as follows:

 - $min\ l(W) = min \sum_{jk}w_jw_kC_{jk}$, where $C_{jk} = (x-\nu_j)\cdot (x-\nu_k)$ subject to $\sum_j w_j =1$

Letting the Lagrangian $L(W, \lambda) = f(W) + \lambda g(W)$, first define $f(W)$, and $g(W)$ according to our minimization problem, and then try to solve the problem analytically. 

### [Optional] Principal components are orthogonal
---

Show that the principal components, i.e. the eigenvectors of the covariance matrix $S=\frac{1}{n}\tilde X\tilde X^T$ are orthogonal. 

## Programming
### Task 1: Implementation of PCA
---
In this task you are asked to implement your own class PCA, that implements a subset of the functions in ```sklearn.decomposition.PCA class```. In particular, your class should implement a fit, transform, and inverse transform method. These should expect as an argument ```X```, a 2 dimensional numpy.ndarray. To compute the eigendecomposition of the covariance matrix of your centered data matrix```X```, you should use ```numpy.linalg.svd```.

In [1]:
import numpy as np
class PCA:
    def __init__(n_components=2):
        self.n_components_ = n_components
        
    def fit(X):
        # todo: implement
        pass
    def transform(X):
        # todo: implement
        pass
    def inverse_transform(X):
        # todo: implement
        pass

## Programming
### [Optional] Extending PCA implementation
---
Extend your PCA implementation, such that
 1. if n_components=None, then all principal components are used
 2. it has attributes ```explained_variance_``` and ```explained_variance_ratio``` such that after fitting, these attributes return the same values as the reference implementation in ```sklearn.decomposition.PCA```.

### [Optional] Task 2: Visualize Mnist in 2D using t-SNE
---
Use sklearn.datasets.load_digits to load the Mnist dataset. Then,
using ```sklearn.manifold.TSNE``` and/or ```sklearn.decomposition.PCA```, try to plot a picture that is close to 
<img src="img\hw.png" alt="Drawing" style="width: 1024px;"/>

In [ ]:
pass